In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import os
import re
import numpy as np
from copy import deepcopy
import random
import scipy.sparse as sp
from time import time

# directory = '../MF/ml-1m'
# ratings = []
# with open(os.path.join(directory, 'ratings.dat'), encoding='latin1') as f:
#     for l in f:
#         user_id, movie_id, rating, timestamp = [int(_) for _ in l.split('::')]
#         ratings.append({
#             'user_id': user_id,
#             'movie_id': movie_id,
#             'rating': rating,
#             'timestamp': timestamp,
#             })
# ratings = pd.DataFrame(ratings)

# ratings = ratings.drop(['timestamp'],axis=1)
# print(ratings)
# # 어떤 비율로 표본 추출을 하고싶은지 df.sample에서 frac을 0~1 사이로 설정
# df = ratings.sample(frac=1).reset_index(drop=True)
# filter_user = deepcopy(df)
# counts = filter_user['user_id'].value_counts()
# filter_user = filter_user[filter_user['user_id'].isin(counts[counts >= 10].index)]

# filtered_df = deepcopy(filter_user)
# counts = filtered_df['movie_id'].value_counts()
# filtered_df = filtered_df[filtered_df['movie_id'].isin(counts[counts >=10].index)]
# filtered_df = filtered_df.reset_index(drop=True)
# print(filtered_df)


class generate_graph(object):
    def __init__(self, path):
        self.path = path  # '../NGCF-PyTorch/Data/ml-1m'
        train_file = path + '/train.txt'
        
        self.neighbor_dict = {}
        self.user, self.item = [], []
        
        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    self.neighbor_dict[int(l[0])] = [int(i) for i in l[1:]]
                    self.user.append(int(l[0]))
        self.user = self.user[:1000]
        
    def jaccard_index(self, u_i, u_j, neighbor_dict):
        u_i_neighbor = self.neighbor_dict[u_i]
        u_j_neighbor = self.neighbor_dict[u_j]
        return len(list(set(u_i_neighbor) & set(u_j_neighbor))) / len(list(set(u_i_neighbor) | set(u_j_neighbor)))

    # 새로운 graph의 node j 에다가 기존 graph의 어떤 node의 neighborhood를 복사할지 zeta에 담기
    def node_copying(self):
        t1 = time()
        zeta = []
        
        for u_j in self.user:
            nor = 0
            zeta_distribution = []
            for u_i in self.user:
                nor+=self.jaccard_index(u_j, u_i, self.neighbor_dict)
            for u_m in self.user:
                zeta_distribution.append(self.jaccard_index(u_j, u_m, self.neighbor_dict) / nor)
            zeta.append(random.choices(self.user, weights=zeta_distribution)[0])
        print('total node copying time cost : ', time() - t1)
        return zeta
    
    def generate_graph(self, epsilon, iteration):
        t2 = time()
        self.epsilon = epsilon
        self.iteration = iteration
        
        generated_node = []
        
        with open(self.path+'/sampled_graph/sampled_graph_'+str(iteration+1), 'w') as f:
            for i in self.user:
                if random.uniform(0,1) < 1-self.epsilon:  # 1-epsilon의 확률로 원래 neighbor 넣기
                    generated_node.append(i)
                else:                                # epsilon의 확률로 zeta에 있는 node의 neighbor로 copy해서 넣기
                    generated_node.append(zeta[i])
                    
                # 만들어지는 새로운 graph를 txt 파일로 저장
                f.write(str(i))
                f.write(' ')
                for j in self.neighbor_dict[generated_node[i]][:-1]:
                    f.write(str(j))
                    f.write(' ')
                f.write(str(self.neighbor_dict[generated_node[i]][-1]))
                f.write('\n')
        print(iteration+1,'-th Graph sampled time cost : ', time() - t2)

In [8]:
t = generate_graph('ml-1m')
zeta = t.node_copying()

zeta
# zeta = node_copying(user)

total node copying time cost :  32.17004346847534


[120,
 908,
 429,
 24,
 277,
 886,
 67,
 201,
 332,
 555,
 795,
 96,
 955,
 421,
 467,
 656,
 250,
 133,
 476,
 495,
 20,
 191,
 397,
 473,
 369,
 298,
 40,
 222,
 455,
 127,
 845,
 625,
 122,
 859,
 207,
 138,
 955,
 571,
 158,
 894,
 563,
 481,
 42,
 592,
 550,
 734,
 46,
 473,
 528,
 230,
 670,
 942,
 396,
 270,
 663,
 145,
 455,
 441,
 58,
 732,
 784,
 224,
 602,
 218,
 954,
 890,
 523,
 466,
 939,
 266,
 457,
 415,
 173,
 810,
 564,
 82,
 934,
 987,
 70,
 725,
 977,
 231,
 505,
 933,
 19,
 21,
 729,
 231,
 215,
 807,
 634,
 875,
 17,
 93,
 577,
 299,
 952,
 483,
 118,
 887,
 748,
 35,
 181,
 227,
 557,
 799,
 606,
 315,
 167,
 918,
 468,
 538,
 272,
 553,
 205,
 551,
 801,
 294,
 577,
 964,
 345,
 695,
 974,
 334,
 557,
 53,
 24,
 909,
 703,
 547,
 273,
 239,
 184,
 523,
 264,
 223,
 880,
 373,
 169,
 517,
 538,
 2,
 162,
 326,
 308,
 700,
 736,
 713,
 962,
 452,
 712,
 345,
 611,
 319,
 730,
 694,
 347,
 112,
 526,
 433,
 924,
 191,
 25,
 376,
 795,
 161,
 326,
 145,
 876,
 887,


In [9]:
# zeta를 한번 만든 후 iteration마다 generate graph하는 방식

for epoch in range(3):
    t.generate_graph(epsilon = 0.01, iteration = epoch)

1 -th Graph sampled time cost :  0.031705617904663086
2 -th Graph sampled time cost :  0.020745277404785156
3 -th Graph sampled time cost :  0.020474910736083984


In [ ]:
# GAT
# 만들어지는 graph마다 embedding 초기화 
# embedding 학습하는 GNN 구조 짜기
# 학습된 embedding 가지고 x_hat (eq.11) 구하고 BPR-OPT를 maximization 시키는 방향으로 학습

In [18]:
# data로 sparse matrix 및 adjacency matrix 만들기

class Data(object):
    def __init__(self, path):
        self.path = path
        
        train_file = path + '/train.txt'
        # path : ml-1m
        
        self.n_users, self.n_items = 0, 0
        self.exist_users = []
        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    self.exist_users.append(uid)
                    self.n_users = max(self.n_users, uid)
                    self.n_items = max(self.n_items, max(items))

        self.n_users+=1
        self.n_items+=1
        print(n_users, n_items)

        self.R = sp.dok_matrix((self.n_users, self.n_items), dtype=np.float32)
        self.train_items = {}

        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l)==0:
                    break
                l = l.strip('\n').split(' ')
                uid, items = int(l[0]), [int(i) for i in l[1:]]

                for i in items:
                    self.R[uid, i] = 1

                self.train_items[uid] = items
                
    def get_adj_mat(self):
        try:
            adj_mat = sp.load_npz(self.path + '/train_adj_mat.npz')
            
        except Exception:
            adj_mat = self.create_adj_mat()
            sp.save_npz(self.path + '/train_adj_mat.npz', adj_mat)
            
        return adj_mat
    
    
    def create_adj_mat(self):
        adj_mat = self.R.todok()
        print('already create adjacency matrix', adj_mat.shape)
        return adj_mat.tocsr()


In [19]:
# data로 sparse matrix 및 adjacency matrix 만들기

# TODO : node copying으로 만들어진 graph들에 대해서 adj matrix 만들고 npz 저장하는 과정 

class sampled_graph_to_matrix(object):
    def __init__(self, path, iteration):
        self.path =path
        self.iteration = iteration
        
        sampled_graph = path + '/sampled_graph/sampled_graph_' + str(iteration+1)
        # path : ../NGCF-PyTorch/Data/ml-1m
        
        self.n_users, self.n_items = 0, 0
        self.exist_users = []
        with open(sampled_graph, 'r') as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    self.exist_users.append(uid)
                    self.n_users = max(self.n_users, uid)
                    self.n_items = max(self.n_items, max(items))

        self.n_users+=1
        self.n_items+=1
        print(self.n_users, self.n_items)

        self.R = sp.dok_matrix((self.n_users, self.n_items), dtype=np.float32)
        self.train_items = {}

        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l)==0:
                    break
                l = l.strip('\n').split(' ')
                uid, items = int(l[0]), [int(i) for i in l[1:]]

                for i in items:
                    self.R[uid, i] = 1

                self.train_items[uid] = items
                
    def get_adj_mat(self):
        try:
            adj_mat = sp.load_npz(self.path + '/s_adj_mat_' + str(self.iteration+1) + '.npz')
            
        except Exception:
            adj_mat = self.create_adj_mat()
            sp.save_npz(self.path + '/s_adj_mat_' + str(self.iteration+1) + '.npz', adj_mat)
            
        return adj_mat
    
    
    def create_adj_mat(self):
        adj_mat = self.R.todok()
        print('already create adjacency matrix', adj_mat.shape)
        return adj_mat.tocsr()


In [20]:
for iteration in range(3):
    sampled_graph = sampled_graph_to_matrix(path='ml-1m', iteration = iteration)
    sampled_graph.get_adj_mat()
    

NameError: name 'n_users' is not defined

In [137]:
class AttenConv(nn.Module):
    def __init__(self):
        super(AttenConv, self).__init__()
    
    def forward(self):
        pass

In [193]:
a = np.matrix([[0,1,2],[1,1,1],[0,0,1]])
adj = np.matrix([[0,0,1],[1,0,0],[1,1,0]])
e = np.matrix([[1,2,3],[4,5,6],[7,8,9]])

print(a,'\n')
print(adj,'\n')
print(e,'\n')

a_jk = np.multiply(a,adj)
print(a_jk, '\n')

np.matmul(a_jk, e)
    
    
    

[[0 1 2]
 [1 1 1]
 [0 0 1]] 

[[0 0 1]
 [1 0 0]
 [1 1 0]] 

[[1 2 3]
 [4 5 6]
 [7 8 9]] 

[[0 0 2]
 [1 0 0]
 [0 0 0]] 



matrix([[14, 16, 18],
        [ 1,  2,  3],
        [ 0,  0,  0]])